In [7]:
import igraph 
import csv
import unicodedata

In [12]:
def CONGAYearAlgorithm(graph,result, Overlap):
    while(graph.ecount()>0):
        edge_betweenness = graph.edge_betweenness(directed=False, cutoff=None, weights=None)
        max_eb = max(edge_betweenness)
        #print(max_eb)
        vertex_betweenness = graph.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)
        max_vb = max(vertex_betweenness)
        #print(max_vb)
        
        if(max_eb>=max_vb):
            edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
            #print(edgesToDelete)
            result.append(edgesToDelete)
            #print(result)
            graph.delete_edges(edgesToDelete)
        else:
            vertexToCopy = vertex_betweenness.index(max_vb);
            #print(vertexToCopy)
            incident = graph.incident(vertexToCopy, mode=ALL)
            #print(incident)
            graph.add_vertices(1)
            graph.vs[graph.vcount()-1]['name'] = graph.vs[vertexToCopy]['name']
            Overlap[graph.vcount()-1] = vertexToCopy
            #print(Overlap)
            
            sum = 0
            for edge in incident:
                #print(graph.es[edge]['weight'])
                sum += graph.es[edge]['weight']

            average = sum/len(incident)
            #print(sum)
            #print(average)
            edges_to_delete = []
            new_vertex_counter = graph.vcount()-1
            allSameWeight = 1
            popWeight = -1
            for edge in incident:
                weight = graph.es[edge]['weight']
                if popWeight != weight and edge != incident[0]:
                    allSameWeight = 0
                if(weight<average):
                    edges_to_delete.append(edge)
                    source = graph.es[edge].source
                    target = graph.es[edge].target
                    #print([source, target])
                    if(target == vertexToCopy):
                        #print([(source, new_vertex_counter)])
                        graph.add_edges([(source, new_vertex_counter)])
                        graph.es[graph.ecount()-1]["weight"]=weight
                    else:
                        #print([(new_vertex_counter, target)])
                        graph.add_edges([(new_vertex_counter, target)])
                        graph.es[graph.ecount()-1]["weight"]=weight
                popWeight = weight
            if allSameWeight == 1:
                #print('All same weight!')
                edges_to_delete.append(edge)
                graph.add_edges([(graph.es[incident[0]].source, new_vertex_counter)])
                graph.es[graph.ecount()-1]["weight"]=weight
                        
            graph.delete_edges(edges_to_delete)
            toAdd = [(vertexToCopy, vertexToCopy)]
            for edge in edges_to_delete:
                toAdd.append([(graph.es[edge].source,graph.es[edge].target)])
            result.append(toAdd)
            #incident2 = graph.incident(vertexToCopy, mode=ALL)
            #print(incident2)
            #print(result)